In [4]:
import pandas as pd
import numpy as np
from PrivBayes import greedy_bayes, construct_noisy_conditional_distributions
from utils import preprocessing, encoding, display_bayesian_network, get_school_list
from model_alpha import model_alpha_synthetic_generator

In [2]:
# Display the dataframe
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [2]:
choices_df = pd.read_excel('data/2021.xlsx', sheet_name='Resultaten')
schools_df = pd.read_excel('data/2021.xlsx', sheet_name='Klassen')

In [5]:
schools = get_school_list(schools_df)

In [6]:
choices_df = preprocessing(choices_df)
choices_df = encoding(choices_df, schools)
choices_df.head()

,Basisschool advies,Voorkeur 1,Voorkeur 2,Voorkeur 3,Voorkeur 4,Voorkeur 5,Voorkeur 6,Voorkeur 7,Voorkeur 8,Voorkeur 9,...,Voorkeur 13,Voorkeur 14,Voorkeur 15,Voorkeur 16,Voorkeur 17,Voorkeur 18,Voorkeur 19,Voorkeur 20,Voorkeur 21,Voorkeur 22
0,havo,135,37,138,104,52,109,102,81,130,...,181,181,181,181,181,181,181,181,181,181
1,havo/vwo,135,64,137,40,5,42,99,102,109,...,181,181,181,181,181,181,181,181,181,181
2,havo/vwo,138,135,102,137,37,68,99,81,20,...,148,109,141,40,181,181,181,181,181,181
3,havo/vwo,135,95,137,138,134,32,104,99,35,...,40,181,181,181,181,181,181,181,181,181
4,havo,135,40,42,109,137,132,30,37,127,...,181,181,181,181,181,181,181,181,181,181


In [8]:
choices_df.columns

Index(['Basisschool advies', 'Voorkeur 1', 'Voorkeur 2', 'Voorkeur 3',
       'Voorkeur 4', 'Voorkeur 5', 'Voorkeur 6', 'Voorkeur 7', 'Voorkeur 8',
       'Voorkeur 9', 'Voorkeur 10', 'Voorkeur 11', 'Voorkeur 12',
       'Voorkeur 13', 'Voorkeur 14', 'Voorkeur 15', 'Voorkeur 16',
       'Voorkeur 17', 'Voorkeur 18', 'Voorkeur 19', 'Voorkeur 20',
       'Voorkeur 21', 'Voorkeur 22'],
      dtype='object')

## Synthetic data using PrivBayes (correlated attribute mode)

In [7]:
edu_types = choices_df['Basisschool advies'].unique().tolist()

In [9]:
from numpy import random
from pandas import DataFrame

vwo = choices_df[choices_df['Basisschool advies'] == 'vwo']
first10_vwo_df = vwo[['Voorkeur 1', 'Voorkeur 2', 'Voorkeur 3', 'Voorkeur 4', 'Voorkeur 5']]

bn = greedy_bayes(first10_vwo_df, k=0, epsilon=0.1 / 2, seed=0)
display_bayesian_network(bn)

epsilon=0.1
conditional_probabilities = construct_noisy_conditional_distributions(bn, first10_vwo_df, epsilon/2)

def get_sampling_order(bn):
    order = [bn[0][1][0]]
    for child, _ in bn:
        order.append(child)
    return order

def bayesian_network_synthetic_generator(n, bn, conditional_probabilities):
    bn_root_attr = bn[0][1][0]
    root_attr_dist = conditional_probabilities[bn_root_attr]
    synthetic_df = DataFrame(columns=get_sampling_order(bn))
    synthetic_df[bn_root_attr] = random.choice(len(root_attr_dist), size=n, p=root_attr_dist)

    for child, parents in bn:
        child_conditional_distributions = conditional_probabilities[child]
        for parents_instance in child_conditional_distributions.keys():
            dist = child_conditional_distributions[parents_instance]
            parents_instance = list(eval(parents_instance))

            # Resolve the error that probabilities do not sum up to 1
            dist = np.asarray(dist).astype('float64')
            dist = dist / np.sum(dist)

            filter_condition = ''
            for parent, value in zip(parents, parents_instance):
                filter_condition += f"(synthetic_df['{parent}']=={value})&"

            filter_condition = eval(filter_condition[:-1])
            
            size = synthetic_df[filter_condition].shape[0]
            if size:
                synthetic_df.loc[filter_condition, child] = random.choice(len(dist), size=size, p=dist)

    synthetic_df[synthetic_df.columns] = synthetic_df[synthetic_df.columns].astype(int)
    return synthetic_df

================ Constructing Bayesian Network (BN) ================
Adding ROOT Voorkeur 7
Adding attribute Voorkeur 5
Adding attribute Voorkeur 8
Adding attribute Voorkeur 6
Adding attribute Voorkeur 4
Adding attribute Voorkeur 3
Adding attribute Voorkeur 9
Adding attribute Voorkeur 2
Adding attribute Voorkeur 10
Adding attribute Voorkeur 1
========================== BN constructed ==========================
Constructed Bayesian network:
    Voorkeur 5  has parents ['Voorkeur 7'].
    Voorkeur 8  has parents ['Voorkeur 5', 'Voorkeur 7'].
    Voorkeur 6  has parents ['Voorkeur 5', 'Voorkeur 8', 'Voorkeur 7'].
    Voorkeur 4  has parents ['Voorkeur 5', 'Voorkeur 8', 'Voorkeur 6', 'Voorkeur 7'].
    Voorkeur 3  has parents ['Voorkeur 5', 'Voorkeur 8', 'Voorkeur 6', 'Voorkeur 4', 'Voorkeur 7'].
    Voorkeur 9  has parents ['Voorkeur 5', 'Voorkeur 6', 'Voorkeur 4', 'Voorkeur 3', 'Voorkeur 7'].
    Voorkeur 2  has parents ['Voorkeur 5', 'Voorkeur 6', 'Voorkeur 4', 'Voorkeur 9', 'Voorkeur 7

KeyboardInterrupt: 

In [ ]:
synthetic_vwo = bayesian_network_synthetic_generator(n=len(vwo), bn=bn, conditional_probabilities=conditional_probabilities)
synthetic_vwo.head()